# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Adamstown,-25.0660,-130.1015,17.63,82,100,14.58,PN,1694791186
1,1,West Island,-12.1568,96.8225,25.99,73,75,6.17,CC,1694791190
2,2,Saint-François,46.4154,3.9054,28.69,39,0,2.57,FR,1694791721
3,3,Port-aux-Français,-49.3500,70.2167,4.64,91,97,15.70,TF,1694791194
4,4,Zemio,5.0314,25.1361,29.71,61,97,1.52,CF,1694791721


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%matplotlib

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    color = "City")

# Display the map
map_plot_1

Using matplotlib backend: QtAgg


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[
    (city_data_df['Max Temp'] >= 21) & 
    (city_data_df['Max Temp'] <= 27) & 
    (city_data_df['Wind Speed'] <= 4.5) &
    (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_conditions_df.count()


City_ID       14
City          14
Lat           14
Lng           14
Max Temp      14
Humidity      14
Cloudiness    14
Wind Speed    14
Country       14
Date          14
dtype: int64

In [5]:
# Drop any rows with null values
cleaned_ideal_conditions_df = ideal_conditions_df.dropna(how='any')
cleaned_ideal_conditions_df.count()

City_ID       14
City          14
Lat           14
Lng           14
Max Temp      14
Humidity      14
Cloudiness    14
Wind Speed    14
Country       14
Date          14
dtype: int64

In [6]:
# Display sample data
cleaned_ideal_conditions_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
104,104,Hancheng,32.5186,112.3522,23.28,67,0,2.35,CN,1694791737
134,134,Holualoa,19.6228,-155.9522,23.91,82,0,3.09,US,1694791253
144,144,Pahrump,36.2083,-115.9839,24.03,49,0,0.89,US,1694791743
154,154,Kailua-Kona,19.6406,-155.9956,25.98,81,0,3.09,US,1694791187
162,162,Jasper,37.2001,-94.3502,24.50,50,0,3.60,US,1694791747


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_ideal_conditions_df.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
104,104,Hancheng,32.5186,112.3522,23.28,67,0,2.35,CN,1694791737,
134,134,Holualoa,19.6228,-155.9522,23.91,82,0,3.09,US,1694791253,
144,144,Pahrump,36.2083,-115.9839,24.03,49,0,0.89,US,1694791743,
154,154,Kailua-Kona,19.6406,-155.9956,25.98,81,0,3.09,US,1694791187,
162,162,Jasper,37.2001,-94.3502,24.50,50,0,3.60,US,1694791747,
204,204,Changde,29.0464,111.6783,23.34,78,0,0.87,CN,1694791754,
212,212,Hamilton,39.1834,-84.5333,22.55,58,0,3.09,US,1694791190,
271,271,Santo Anastácio,-21.8383,-51.6103,24.20,27,0,1.45,BR,1694791766,
343,343,Grantsville,40.5999,-112.4644,21.09,36,0,1.54,US,1694791778,
351,351,Cologne,50.9333,6.9500,25.03,48,0,2.06,DE,1694791202,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'api_key': geoapify_key,
    'categories': 'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Hancheng - nearest hotel: No hotel found
Holualoa - nearest hotel: Kona Hotel
Pahrump - nearest hotel: Holiday Inn Express & Suites
Kailua-Kona - nearest hotel: Kona Seaside Hotel
Jasper - nearest hotel: Econo Lodge
Changde - nearest hotel: Sheraton Changde Wuling Hotel
Hamilton - nearest hotel: North Vista Manor
Santo Anastácio - nearest hotel: No hotel found
Grantsville - nearest hotel: No hotel found
Cologne - nearest hotel: Wasserturm Hotel Cologne
Bhamo - nearest hotel: Winlight Hotel
Haiku-Pauwela - nearest hotel: Inn At Mama's Fish House
Türkmenbaşy - nearest hotel: Türkmenbaşy Oteli
Mŭ’minobod - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
104,104,Hancheng,32.5186,112.3522,23.28,67,0,2.35,CN,1694791737,No hotel found
134,134,Holualoa,19.6228,-155.9522,23.91,82,0,3.09,US,1694791253,Kona Hotel
144,144,Pahrump,36.2083,-115.9839,24.03,49,0,0.89,US,1694791743,Holiday Inn Express & Suites
154,154,Kailua-Kona,19.6406,-155.9956,25.98,81,0,3.09,US,1694791187,Kona Seaside Hotel
162,162,Jasper,37.2001,-94.3502,24.50,50,0,3.60,US,1694791747,Econo Lodge
204,204,Changde,29.0464,111.6783,23.34,78,0,0.87,CN,1694791754,Sheraton Changde Wuling Hotel
212,212,Hamilton,39.1834,-84.5333,22.55,58,0,3.09,US,1694791190,North Vista Manor
271,271,Santo Anastácio,-21.8383,-51.6103,24.20,27,0,1.45,BR,1694791766,No hotel found
343,343,Grantsville,40.5999,-112.4644,21.09,36,0,1.54,US,1694791778,No hotel found
351,351,Cologne,50.9333,6.9500,25.03,48,0,2.06,DE,1694791202,Wasserturm Hotel Cologne


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
map_plot_2 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    hover_cols=["Hotel Name", "Country"],
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size=30,
    scale=0.7,
    color="City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)